In [21]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [7]:
base_url="http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules="


In [23]:
URL = "http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules="
county_pagepage = requests.get(URL)

soup= BeautifulSoup(page.text)

In [82]:
selectors=soup.find_all("select")
for s in selectors:
    if "megye" in s["onchange"]:
        county_opts = {o.text: o["value"] for o in s.contents if "Válasszon" not in o.text}
        break

county_opts
        

{'Bács-Kiskun Megye': 'del-alfoldi-regio-1/bacs-kiskun-megye-1',
 'Baranya Megye': 'del-dunantuli-regio-2/baranya-megye-2',
 'Békés Megye': 'del-alfoldi-regio-1/bekes-megye-3',
 'Borsod-Abaúj-Zemplén Megye': 'eszak-magyarorszagi-regio-4/borsod-abauj-zemplen-megye-4',
 'Budapest Főváros': 'kozep-magyarorszagi-regio-6/budapest-fovaros-5',
 'Csongrád Megye': 'del-alfoldi-regio-1/csongrad-megye-6',
 'Fejér Megye': 'kozep-dunantuli-regio-5/fejer-megye-7',
 'Győr-Moson-Sopron Megye': 'nyugat-dunantuli-regio-7/gyor-moson-sopron-megye-8',
 'Hajdú-Bihar Megye': 'eszak-alfoldi-regio-3/hajdu-bihar-megye-9',
 'Heves Megye': 'eszak-magyarorszagi-regio-4/heves-megye-10',
 'Jász-Nagykun-Szolnok Megye': 'eszak-alfoldi-regio-3/jasz-nagykun-szolnok-megye-11',
 'Komárom-Esztergom Megye': 'kozep-dunantuli-regio-5/komarom-esztergom-megye-12',
 'Nógrád Megye': 'eszak-magyarorszagi-regio-4/nograd-megye-13',
 'Pest Megye': 'kozep-magyarorszagi-regio-6/pest-megye-14',
 'Somogy Megye': 'del-dunantuli-regio-2/so

In [90]:
ks = ['Bács-Kiskun Megye']

all_towns={}
for k in county_opts:
    county_URL=URL+county_opts[k]
    print(county_URL)
    
    county_page = requests.get(county_URL)
    county_soup= BeautifulSoup(county_page.text)

    selectors=county_soup.find_all("select")
    for s in selectors:
        if "telepules" in s["onchange"]:
            opts = {o.text: o["value"] for o in s.contents if "Válasszon" not in o.text}
            break

    all_towns.update(opts)
    time.sleep(1)


    

http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=del-alfoldi-regio-1/bacs-kiskun-megye-1
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=del-dunantuli-regio-2/baranya-megye-2
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=del-alfoldi-regio-1/bekes-megye-3
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=eszak-magyarorszagi-regio-4/borsod-abauj-zemplen-megye-4
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=kozep-magyarorszagi-regio-6/budapest-fovaros-5
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=del-alfoldi-regio-1/csongrad-megye-6
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=kozep-dunantuli-regio-5/fejer-megye-7
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adatbazis/?telepules=nyugat-dunantuli-regio-7/gyor-moson-sopron-megye-8
http://xn--tosz-5qa.hu/szolgaltatasaink/onkormanyzati-adat

In [96]:
with open("toosz_towns.json","w") as f:
    json.dump(all_towns,f)

In [3]:
with open("toosz_towns.json") as f:
    all_towns=json.load(f)

In [22]:
ks = list(all_towns.keys())[:100]

df = pd.DataFrame(columns=['Polgármester e-mail', 'Polgármester', 'Cím', 'Önkormányzat telefon',
       'Önkormányzat e-mail'])

for k in ks:
    town_url=base_url+all_towns[k]
    print(k)
    
    town_page = requests.get(town_url)
    town_soup= BeautifulSoup(town_page.text)
    town_soup.find_all("col_md_6")

    col_md_6s=town_soup.find_all("div", {"class": "col-md-6"})

    town_info={}
    for c in col_md_6s:
        if "önkormányzat" in str(c.contents[0]).lower():
            for line in c.contents[0].contents:
                if "Cím:" in line:
                    town_info['Cím']=line.replace("Cím: ","")
                elif "Telefon" in line:
                    town_info['Önkormányzat telefon']=line.replace("Telefon: ","")
                elif "mailto:" in str(line):
                    town_info['Önkormányzat e-mail']=line.text

        if "polgármester" in str(c.contents[0]).lower():
            for line in c.contents[0].contents:
                if "Cím:" in line:
                    town_info['Cím']=line.replace("Cím: ","")
                elif "Telefon" in line:
                    town_info['Polgármester telefon']=line.replace("Telefon: ","")
                elif "mailto:" in str(line):
                    town_info['Polgármester e-mail']=line.text
            
            town_info['Polgármester']=c.contents[0].contents[3]
    town_series=pd.Series(town_info,name=k)
    df.loc[k]=town_info
    time.sleep(1)
        

Ágasegyháza
Akasztó
Apostag
Bácsalmás
Bácsbokod
Bácsborsód
Bácsszentgyörgy
Bácsszőlős
Baja
Ballószög
Balotaszállás
Bátmonostor
Bátya
Bócsa
Borota
Bugac
Bugacpusztaháza
Császártöltés
Csátalja
Csávoly
Csengőd
Csikéria
Csólyospálos
Dávod
Drágszél
Dunaegyháza
Dunafalva
Dunapataj
Dunaszentbenedek
Dunatetétlen
Dunavecse
Dusnok
Érsekcsanád
Érsekhalma
Fajsz
Felsőlajos
Felsőszentiván
Foktő
Fülöpháza
Fülöpjakab
Fülöpszállás
Gara
Gátér
Géderlak
Hajós
Harkakötöny
Harta
Helvécia
Hercegszántó
Homokmégy
Imrehegy
Izsák
Jakabszállás
Jánoshalma
Jászszentlászló
Kalocsa
Kaskantyú
Katymár
Kecel
Kecskemét
Kelebia
Kéleshalom
Kerekegyháza
Kiskőrös
Kiskunfélegyháza
Kiskunhalas
Kiskunmajsa
Kisszállás
Kömpöc
Kunadacs
Kunbaja
Kunbaracs
Kunfehértó
Kunpeszér
Kunszállás
Kunszentmiklós
Ladánybene
Lajosmizse
Lakitelek
Madaras
Mátételke
Mélykút
Miske
Móricgát
Nagybaracska
Nemesnádudvar
Nyárlőrinc
Ordas
Öregcsertő
Orgovány
Páhi
Pálmonostora
Petőfiszállás
Pirtó
Rém
Solt
Soltszentimre
Soltvadkert
Sükösd
Szabadszállás


In [26]:
df.to_excel("polgik_100.xlsx")

In [210]:
df.index

Index(['Polgármester e-mail', 'Polgármester', 'Cím', 'Önkormányzat telefon',
       'Önkormányzat e-mail'],
      dtype='object')

In [190]:
col_md_6s=town_soup.find_all("div", {"class": "col-md-6"})

town_info={}
for c in col_md_6s:
    if "önkormányzat" in str(c.contents[0]).lower():
        for line in c.contents[0].contents:
            if "Cím:" in line:
                town_info['Cím']=line.replace("Cím: ","")
            elif "Telefon" in line:
                town_info['Önkormányzat telefon']=line.replace("Telefon: ","")
            elif "mailto:" in str(line):
                town_info['Önkormányzat e-mail']=line.text

    if "polgármester" in str(c.contents[0]).lower():
        for line in c.contents[0].contents:
            if "Cím:" in line:
                town_info['Cím']=line.replace("Cím: ","")
            elif "Telefon" in line:
                town_info['Polgármester telefon']=line.replace("Telefon: ","")
            elif "mailto:" in str(line):
                town_info['Polgármester e-mail']=line.text
        
        town_info['Polgármester']=c.contents[0].contents[3]
print(town_info)
    

{'Polgármester e-mail': 'furedi.janos@t-online.hu', 'Polgármester': 'Füredi János', 'Cím': '6076 Ágasegyháza, Szent István tér 1.', 'Önkormányzat telefon': '(76)571-009', 'Önkormányzat e-mail': 'pmstrh@t-online.hu'}


In [187]:
c=col_md_6s[0]

town_info={}


if "önkormányzat" in str(c.contents[0]).lower():
    for line in c.contents[0].contents:
        if "Cím:" in line:
            town_info['Cím']=line.replace("Cím: ","")
        elif "Telefon" in line:
            town_info['Önkormányzat telefon']=line.replace("Telefon: ","")
        elif "mailto:" in str(line):
            town_info['Önkormányzat e-mail']=line.text

if "polgármester" in str(c.contents[0]).lower():
    for line in c.contents[0].contents:
        if "Cím:" in line:
            town_info['Cím']=line.replace("Cím: ","")
        elif "Telefon" in line:
            town_info['Önkormányzat telefon']=line.replace("Telefon: ","")
        elif "mailto:" in str(line):
            town_info['Önkormányzat e-mail']=line.text
    
    town_info['Polgármester']=c.contents[0].contents[3]
town_info

{'Önkormányzat e-mail': 'furedi.janos@t-online.hu',
 'Polgármester': 'Füredi János'}

In [186]:
c.contents[0].contents[3]

'Füredi János'